In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )
y["train"] = np.load("./train_label_save.npy").squeeze()
X["dev"] = np.load("./test_feature_save.npy")
y["dev"] = np.load("./test_label_save.npy").squeeze()
X_text_train = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_tweet/X_train_text.npy")
X_text_dev = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_tweet/X_test_text.npy")
X["train"].shape, X["dev"].shape, y["dev"].shape

((18000, 768), (1000, 768), (1000,))

In [2]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.758

In [3]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((18000, 769), (1000, 769))

In [4]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 10 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [5]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1000, 18000)

In [17]:
app_k = np.array(np.load("appro_ks_IP.npy", allow_pickle=True).squeeze())
pred = np.load("old_predictions.npy", allow_pickle=True)
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if app_k[i] == None:
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

4172
78
465
544
468
320
368
6
107
47
962
26
8
645
None
27
1053
255
271
None
1040
None
1912
776
4888
79
705
261
None
259
76
113
135
93
446
41
2662
1541
7539
154
3746
1
50
1161
384
9
380
229
38
281
328
320
78
None
828
285
289
17
36
11
79
7241
None
228
410
194
40
713
3360
314
12
22
15
1766
934
530
1194
85
13
203
74
62
77
3165
35
986
1424
779
193
910
3129
1376
150
157
2246
1119
3
221
100
109
57
69
70
196
406
1
61
133
91
614
42
437
10
26
766
55
3551
267
1552
4617
239
108
16
18
669
None
99
61
234
77
393
3346
14
639
1009
1015
1583
None
18
405
189
301
None
2594
70
64
932
78
271
2205
None
1913
23
1081
518
None
109
5340
105
28
208
1401
91
63
2738
342
1357
4443
36
1053
64
137
1022
126
484
320
1167
1091
174
595
51
460
2485
173
263
81
482
14
114
747
97
532
1982
1768
2
14
59
1654
1258
291
976
223
3035
280
2924
25
81
84
30
1561
1
678
988
276
12
626
261
395
1932
588
228
569
83
1327
632
None
None
148
155
36
54
1907
61
None
415
187
104
None
39
6916
2113
674
20
None
599
3838
None
147
3427
1342
None
1619


In [18]:
np.median(app_k_nonon)

377.0

In [19]:
sort_dev_k = np.array(app_k_nonon, dtype=int).argsort()
sort_dev_k

array([105, 950,  41, 210, 194, 902, 339, 412,  96, 276, 606, 464, 650,
         7, 816, 733, 296, 519, 456, 537,  12, 918, 756, 316, 278,  45,
       446, 765, 974, 112, 677,  59, 990, 214, 972, 882, 910,  70, 300,
        78, 535, 273, 548, 132, 187, 292, 195,  72, 319, 350, 291, 122,
       908, 294, 632,  57, 138, 628, 361, 123, 571, 242, 378, 682, 656,
        71, 324, 152, 670, 880, 386, 623, 994, 286, 277, 205, 560, 113,
       499,  11, 305, 934, 825, 769, 646,  15, 984, 736, 159, 983, 468,
       208, 687, 611, 263, 947, 749, 712, 857,  84, 168, 358,  58, 981,
       229, 366, 870,  48, 573, 308, 238, 502, 969, 503, 483, 957,  66,
       807,  35, 110, 839, 959, 438, 318, 264, 755, 770, 871,   9, 952,
       521, 425, 399, 338, 779,  42, 269, 180, 912, 649, 230, 440, 549,
       626, 433, 514, 115, 342, 100, 629, 445, 523, 637, 940, 196, 432,
       493, 976, 332, 232, 127, 106, 846,  81, 812, 163, 477, 145, 325,
       170, 690, 895, 848, 956, 774, 669, 761, 101, 313, 293, 63

In [20]:
app_k_nonon[253]

1498

In [21]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [22]:
new_predictions = np.load("new_predictions.npy", allow_pickle=True)

In [23]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if k == None:
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id]) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1
k 1
k 1
k 1
k 2
k 2
k 2
k 2
k 3
k 3
k 4
k 4
k 4
k 6
k 6
k 7
k 8
k 8
k 8
k 8
k 8
k 9
k 9
k 9
k 9
k 9
k 9
k 9
k 9
k 10
k 11
k 11
k 12
k 12
k 12
k 12
k 12
k 12
k 12
k 13
k 13
k 13
k 14
k 14
k 14
k 14
k 14
k 15
k 15
k 15
k 15
k 16
k 16
k 16
k 17
k 17
k 18
k 18
k 18
k 18
k 20
k 20
k 20
k 21
k 22
k 22
k 22
k 23
k 23
k 23
k 23
k 23
k 24
k 25
k 25
k 25
k 26
k 26
k 26
k 26
k 26
k 27
k 27
k 27
k 27
k 27
k 28
k 28
k 28
k 28
k 28
k 30
k 31
k 32
k 33
k 34
k 35
k 35
k 35
k 35
k 36
k 36
k 36
k 36
k 36
k 37
k 38
k 38
k 39
k 39
k 39
k 39
k 39
k 40
k 40
k 40
k 40
k 40
k 41
k 42
k 43
k 45
k 45
k 45
k 46
k 46
k 46
k 47
k 47
k 47
k 48
k 48
k 50
k 50
k 50
k 50
k 51
k 51
k 52
k 53
k 54
k 54
k 54
k 55
k 55
k 55
k 55
k 56
k 57
k 58
k 59
k 59
k 59
k 59
k 59
k 60
k 60
k 60
k 61
k 61
k 61
k 61
k 62
k 62
k 63
k 63
k 63
k 64
k 64
k 64
k 66
k 66
k 66
k 66
k 66
k 67
k 68
k 69
k 69
k 69
k 69
k 70
k 70
k 72
k 73
k 73
k 74
k 74
k 74
k 74
k 74
k 74
k 75
k 75
k 75
k 76
k 76
k 77
k 77
k 77
k 77
k 78
k 78
k 78
k 78
k 78

In [24]:
import csv

with open('CSV_tweet_bertlin_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [15]:
ls

CSV_essay_bertlin_alg1.ipynb  new_predictions.npy
CSV_speech_bertlin_alg1.csv   old_predictions.npy
Smallest_k.ipynb              run_retrain.sh
Smallest_k.py                 run_tagger.sh
Use_approK.ipynb              save_feature.py
__pycache__/                  save_feature.sh
appro_ks_IP.npy               test_feature_save.npy
bert_util.py                  test_label_save.npy
essay_data/                   train_feature_save.npy
model_save_feature/           train_label_save.npy


In [16]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_speech'